## Machine Learning & AI

![Colegio Bourbaki](./Images/Bourbaki.png)

### Temporal Fusion Transformerr

In [ ]:
#!pip install --upgrade torch pytorch-lightning pytorch-forecasting torchdata optuna

Vamos a utilizar la libreria pytorch-forecasting de la cual importaremos el modelo TFT. También puede utilizarse el modelo de la libreria neuralforecast de la empresa NIXTLA. Ambas se apalancan PyTorch Lightninggggg

Links:

* **Pytorch Forecasting:** https://pytorch-forecasting.readthedocs.io/en/stable/index.html
* **Neural Forecast:** https://nixtlaverse.nixtla.io/neuralforecast/docs/getting-started/introduction.html
* **Pytorch Lightning:** https://lightning.ai/docs/pytorch/stable/

Librerias

In [ ]:
# Data Analysis
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt

# Pytorch Lighting
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.tuner import Tuner

# Pytorch Forcasting
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import  QuantileLoss, MAE, RMSE, SMAPE, MAPE, MASE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# Utils
import pickle
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error

No vamos a explicar nuevamente el contexto del problema, iremos directamente a la solución del problema ya planteado. La primera parte del código es similar, por lo tanto, muchas celdas no serán descriptas.

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
device

Carga de Datos

In [ ]:
x_train = pd.read_csv('./Data/SNCF/hard/Xtrain_hgcGIrA.csv')

In [ ]:
index = x_train.index

In [ ]:
index

In [ ]:
x_train.info()

In [ ]:
df_x_train = x_train.copy(deep=True)

In [ ]:
y_train = pd.read_csv(
    "/home/pdconte/Desktop/Colegio_Bourbaki/ML_AI/Semana2/Data/SNCF/hard/Ytrain_yL5OjS4.csv"
)

In [ ]:
df_y_train = y_train[['p0q0']]
df_y_train = df_y_train.set_index(index, drop=True)

In [ ]:
df_x_train

In [ ]:
df_x_train['train'].unique()

In [ ]:
df_x_train['train'].nunique()

In [ ]:
df_x_train["station"].unique()

In [ ]:
df_x_train["station"].nunique()

In [ ]:
df_train = pd.concat(
    [
        df_x_train,
        df_y_train
    ],
    axis=1,
)

In [ ]:
df_train

In [ ]:
df_train.info()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train['hour'].value_counts(dropna=False)

In [ ]:
#Fill 'hour' nan valuee with -1
df_train['hour'] = df_train['hour'].fillna(-1)

In [ ]:
df_train['hour'].value_counts(dropna=False)

In [ ]:
df_train['hour'] = df_train['hour'].apply(lambda x: '10:00:00' if x == -1 else x)

In [ ]:
df_train['hour'].value_counts(dropna=False)

In [ ]:
df_train["date"] = pd.to_datetime(df_train["date"], format="mixed")
df_train["hour"] = pd.to_datetime(df_train["hour"], format="%H:%M:%S").dt.hour

In [ ]:
# Extract day of week and month as features
df_train['day_of_week'] = df_train['date'].dt.dayofweek
df_train['month'] = df_train['date'].dt.month

In [ ]:
df_train['train'] = df_train['train'].astype("str")
df_train["station"] = df_train["station"].astype("str")
df_train["way"] = df_train["way"].astype("str")
df_train["composition"] = df_train["composition"].astype("str")
df_train = df_train.rename(columns={"train": "train_id"})

In [ ]:
df_train.isna().sum()

In [ ]:
df_train.info()

Vamos a crear una nueva columna llamada time_idx donde aisgnaremos cada fila en datos un índice basado en la posición de su valor de fecha (date) dentro dela variable de una lista (lista).

In [ ]:
# First, sort the data by date and hour to ensure correct temporal ordering
df_train = df_train.sort_values(by=["date", "hour"]).reset_index(drop=True)

# Create a unique time index within each group of (train, station)
df_train["time_idx"] = df_train.groupby(["train_id", "station"]).cumcount()

In [ ]:
df_train.info()

El time_idx es un componente crucial a la hora de preparar datos para modelos de series temporales, especialmente en el contexto de la TFT y la biblioteca pytorch-forecasting. Sirve para varios propósitos importantes:

* Orden temporal: Proporciona un índice numérico secuencial que representa el orden de los pasos temporales en los datos.
* Alinear secuencias: Ayuda a alinear correctamente las secuencias para las partes codificadora (encoder) y decodificadora (decoder) del modelo.
* Manejo de múltiples series temporales: Cuando se manejan múltiples series temporales (por ejemplo, diferentes trenes), asegura que los pasos temporales estén correctamente ordenados dentro de cada grupo.

**¿Por qué no utilizar directamente fechas o marcas de tiempo?**

Aunque las fechas y las marcas de tiempo contienen información temporal, pueden no ser adecuadas para indexar el tiempo en el modelo porque:
* Intervalos irregulares: Las fechas pueden no estar espaciadas por igual (por ejemplo, fines de semana, días festivos), lo que puede complicar el modelado secuencial.
* No numéricos: Los modelos requieren índices numéricos para los pasos temporales para realizar cálculos de forma eficiente.
* Complejidad: El cálculo de diferencias temporales y la alineación de secuencias mediante fechas pueden introducir una complejidad innecesaria.

In [ ]:
df_train

Comenzaremos a utilizar Pytorch Forecasting para resolver nuestro problema:

El siguiente paso es convertir el marco de datos en un PyTorch Forecasting TimeSeriesDataSet. Aparte de indicar al conjunto de datos qué características son categóricas frente a continuas y cuáles son estáticas frente a variables en el tiempo, también tenemos que decidir cómo normalizamos los datos. Aquí, normalizamos la escala de cada serie temporal por separado e indicamos que los valores son siempre positivos. Generalmente, se prefiere el EncoderNormalizer, que escala dinámicamente en cada secuencia del codificador a medida que se entrena, para evitar el sesgo look-ahead inducido por la normalización. Sin embargo, puede aceptar el sesgo de anticipación si tiene problemas para encontrar una normalización razonablemente estable, por ejemplo, porque hay muchos ceros en sus datos. O si espera una normalización más estable en la inferencia. En este último caso, se asegura de que no aprende saltos "raros" que no estarán presentes cuando ejecute la inferencia, entrenando así en un conjunto de datos más realista.

Primero veamos un poco de estadisticas del time_idx

In [ ]:
# Count the range of time_idx for each train_id and station
time_idx_stats = df_train.groupby(["train_id", "station"])["time_idx"].agg(["min", "max", "count"])
time_idx_stats["range"] = time_idx_stats["max"] - time_idx_stats["min"]
print(time_idx_stats, '\n')

# Summary statistics to see the distribution of time_idx ranges
print("Time_idx range statistics across groups:")
print(time_idx_stats["range"].describe())

In [ ]:
time_idx_stats['range'].hist()
plt.show()

In [ ]:
# Count the number of stops (unique time_idx) per train and station
stops_per_train_station = df_train.groupby(["train_id", "station"])["time_idx"].nunique()
print("Stops per train-station pair:")
print(stops_per_train_station.describe())

In [ ]:
stops_per_train_station.hist()
plt.show()

In [ ]:
df_train['time_idx'].value_counts()

In [ ]:
# Plot occupancy rates over time_idx for a specific train and station
sample_train_id = np.random.choice(df_train["train_id"].unique())  # Choose an example train randomly
sample_station = np.random.choice(df_train["station"].unique())  # Choose an example station randomly

sample_data = df_train[(df_train["train_id"] == sample_train_id) & (df_train["station"] == sample_station)]
plt.plot(sample_data["time_idx"], sample_data["p0q0"], marker="o")
plt.xlabel("Time Index (time_idx)")
plt.ylabel("Occupancy Rate (p0q0)")
plt.title(f"Occupancy Rate Over Time for Train {sample_train_id} at Station {sample_station}")
plt.show()

Observaciones de la Exploración de Datos

* Rango de time_idx (Contexto Histórico):

La mayoría de las combinaciones de tren-estación tienen entre 48 y 64 pasos de tiempo (time_idx) en el conjunto de datos.
El valor mediano (percentil 50) para el range es 57, lo que indica que la mayoría de los pares tren-estación tienen alrededor de 57 paradas.

* Longitud Típica de Secuencia (Paradas por Par Tren-Estación): El promedio de paradas es aproximadamente 57, y el máximo es 65.

In [ ]:
# Define the lag columns with structural NaNs
lag_columns = ["p1q0", "p2q0", "p3q0", "p0q1", "p0q2", "p0q3"]

# Fill NaNs with -1 for these columns
df_train[lag_columns] = df_train[lag_columns].fillna(-1)

In [ ]:
df_train.columns

In [ ]:
max_encoder_length = 6 
max_prediction_length = 3
training_cutoff = df_train["time_idx"].max() - max_prediction_length

In [ ]:
training_cutoff

El parámetro *max_encoder_length* especifica la longitud de la secuencia de entrada (también conocida como secuencia codificadora) que el modelo utiliza para hacer predicciones. En otras palabras, determina cuántos pasos temporales pasados tendrá en cuenta el modelo a la hora de pronosticar valores futuros.

En la arquitectura del Transformador de Fusión Temporal, los datos se procesan de secuencia a secuencia:
* Codificador: Procesa los datos históricos hasta el paso temporal actual. La longitud de esta secuencia viene determinada por max_encoder_length.
* Decodificador: Genera predicciones para futuros pasos temporales. La longitud del horizonte de predicción viene determinada por max_prediction_length.

**¿Cómo afecta *max_encoder_length* al modelo?**
* Ventana de contexto: Al establecer *max_encoder_length*, se define el tamaño de la ventana de contexto que tiene el modelo para aprender patrones a partir de datos históricos.
* Captura de dependencias temporales: Un *max_encoder_length* mayor permite al modelo captar dependencias y tendencias a largo plazo en los datos.
* Complejidad computacional: Aumentar *max_encoder_length* incrementa la carga computacional y el uso de memoria ya que el modelo tiene que procesar secuencias más largas.

In [ ]:
training_dataset = TimeSeriesDataSet(
    df_train[df_train["time_idx"] <= training_cutoff],
    time_idx="time_idx",
    target="p0q0",
    group_ids=["train_id", "station"],
    static_categoricals=["train_id", "station"],
    time_varying_known_categoricals=[],
    time_varying_known_reals=['time_idx', 'month', 'hour', "way", "composition", 'day_of_week'], 
    time_varying_unknown_categoricals=[],
    time_varying_unknown_reals=["p0q1", "p0q2", "p0q3", "p1q0", "p2q0", "p3q0"], # you can pass date in YYYYMMDD format, not timestamp
    #min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    #min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    variable_groups={},  # group of categorical variables can be treated as one variable
    target_normalizer=GroupNormalizer(
        groups=["train_id", "station"], transformation="softplus", scale_by_group=True
    ),  # use softplus and normalize by group
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
    #randomize_length=True
)

Identificar tipos de variables
* *time_varying_known_reals*: Variables conocidas de antemano (por ejemplo, características de fecha).
* *time_varying_unknown_reals*: Variables que cambian con el tiempo pero que no se conocen de antemano (por ejemplo, variables de retardo).
* *static_categoricals*: Variables que son constantes para una serie temporal (por ejemplo, ID del tren ).

Link: https://pytorch-forecasting.readthedocs.io/en/stable/api/pytorch_forecasting.data.timeseries.TimeSeriesDataSet.html#pytorch_forecasting.data.timeseries.TimeSeriesDataSet

In [ ]:
training_dataset.get_parameters()

In [ ]:
# create validation set (predict=True) which means to predict the last max_prediction_length points in time for each series
validation_dataset = TimeSeriesDataSet.from_dataset(
    training_dataset, 
    df_train, 
    predict=True, 
    stop_randomization=True,
)

In [ ]:
# create dataloaders for model
batch_size = 64
train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=6)
val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size, num_workers=6)

Aquí estamos configurando un conjunto de datos para entrenar y validar un modelo de series temporales utilizando la biblioteca pytorch-forecasting, que es popular para tareas de predicción en series temporales con PyTorch.

A continuación, se detalla paso a paso lo que se está realizando:

  * Definición de Parámetros: Se establecen dos parámetros, max_prediction_length y max_encoder_length, que definen respectivamente la longitud máxima de la predicción y la longitud máxima del codificador (historia o contexto utilizado para la predicción).

  * Cálculo del Punto de Corte para el Entrenamiento: Se calcula training_cutoff para determinar hasta qué punto en el tiempo se incluirán los datos en el conjunto de entrenamiento. Esto se hace para asegurar que haya suficientes datos hacia el final del conjunto de datos para validar el modelo.

  * Creación del Conjunto de Datos para Entrenamiento (TimeSeriesDataSet):
      Se filtran los datos para incluir solo aquellos puntos en el tiempo que son iguales o anteriores al training_cutoff.
      Se especifican varias configuraciones importantes para el conjunto de datos, como los índices de tiempo (time_idx), la variable objetivo (target), los identificadores de grupo (group_ids), las longitudes mínima y máxima del codificador, y la longitud de la predicción.
      Se definen configuraciones adicionales como categorías estáticas y reales, variables conocidas y desconocidas tanto categóricas como reales, y se establece un normalizador de grupo para la variable objetivo.
      Se añaden características adicionales como el índice de tiempo relativo, las escalas del objetivo y la longitud del codificador.

  * Creación del Conjunto de Datos para Validación (TimeSeriesDataSet.from_dataset):
      Se crea un conjunto de datos para validación a partir del conjunto de entrenamiento, utilizando la totalidad de los datos pero configurando el conjunto para predecir los últimos max_prediction_length puntos para cada serie temporal.
      Esto permite evaluar cómo el modelo predice los datos "futuros" basándose en el conocimiento "pasado".

  * Creación de DataLoader para Entrenamiento y Validación:
      Se crean DataLoader para ambos conjuntos, permitiendo la carga eficiente de datos durante el entrenamiento y la validación del modelo en mini-lotes de tamaño batch_size.

Este proceso es típico en la preparación de datos para el entrenamiento de modelos de series temporales, permitiendo un manejo eficiente de diferentes longitudes de series, incorporación de variables explicativas y ajuste de parámetros específicos para la predicción temporal. La biblioteca pytorch-forecasting facilita la manipulación de series temporales complejas y la implementación de modelos de deep learning para su predicción.

Para determinar el valor adecuado de max_prediction_length para tu conjunto de datos, debes considerar el objetivo de tu análisis de series temporales y la naturaleza de los datos. El valor de max_prediction_length no se deriva directamente de los datos en sí, sino que es una elección que depende de cuánto en el futuro deseas predecir basado en la estructura temporal de tu conjunto de datos.

Ahora, pasaremos al modelo:

Vamos a calcular un benckmark base al cual deberíamos superar:

In [ ]:
# calculate baseline errors, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(train_dataloader)])
baseline_predictions = Baseline().predict(train_dataloader, return_y=False, mode='prediction')

In [ ]:
baseline_predictions

In [ ]:
mae_baseline_predictions = MAE()(baseline_predictions.cpu(), actuals)
mape_baseline_predictions = MAPE()(baseline_predictions.cpu(), actuals)
smape_baseline_predictions = SMAPE()(baseline_predictions.cpu(), actuals)
rmse_baseline_predictions = RMSE()(baseline_predictions.cpu(), actuals)

In [ ]:
# Print baseline errors
print("MAE:", mae_baseline_predictions.item())
print("MAPE:", mape_baseline_predictions.item())
print("SMAPE:", smape_baseline_predictions.item())
print("RMSE:", rmse_baseline_predictions.item())
print("MSE:", (rmse_baseline_predictions.item())**2)

**Temporal Fusion Transformers**

Las previsiones multihorizonte a menudo contienen una compleja mezcla de variables de entrada - incluidas covariables estáticas (es decir, invariables en el tiempo), variables de entrada futuras conocidas y otras series temporales exógenas que solo se observan en el pasado - sin ninguna información previa sobre cómo interactúan con el objetivo. Se han propuesto varios métodos de aprendizaje profundo, pero suelen ser modelos de "caja negra" que no arrojan luz sobre cómo utilizan toda la gama de entradas presentes en escenarios prácticos.

El Transformador de Fusión Temporal (TFT), una arquitectura basada en la atención que combina la previsión multihorizonte de alto rendimiento con una visión interpretable de la dinámica temporal.

Para aprender las relaciones temporales a diferentes escalas, el TFT utiliza capas recurrentes para el procesamiento local y capas interpretables de autoatención para las dependencias a largo plazo.

TFT utiliza componentes especializados para seleccionar las características relevantes y una serie de capas de compuerta para suprimir los componentes innecesarios, lo que permite un alto rendimiento en una amplia gama de escenarios.

* Link: Temporal Fusion Transformers for Interpretable Multi-horizon Time Series Forecasting - https://arxiv.org/pdf/1912.09363.pdf
* Link: https://pytorch-forecasting.readthedocs.io/en/stable/api/pytorch_forecasting.models.temporal_fusion_transformer.TemporalFusionTransformer.html#pytorch_forecasting.models.temporal_fusion_transformer.TemporalFusionTransformer


Instanciamos el modelo:

In [ ]:
# configure network and trainer
pl.seed_everything(42)

trainer = pl.Trainer(
    accelerator="gpu", #cpu
    devices=1,
    gradient_clip_val=0.1, # clipping gradients is a hyperparameter and important to prevent divergance of the gradient for recurrent neural networks
)

tft = TemporalFusionTransformer.from_dataset(
    training_dataset,  
    learning_rate=0.03, # not meaningful for finding the learning rate but otherwise very important
    hidden_size=16,  # most important hyperparameter apart from learning rate
    attention_head_size=1,  # number of attention heads. Set to up to 4 for large datasets
    dropout=0.1,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=8,  # set to <= hidden_size
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    reduce_on_plateau_patience=4, # reduce learning rate if no improvement in validation loss after x epochs
    optimizer='adam'
)

In [ ]:
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# find optimal learning rate
res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
    max_lr=1,
    min_lr=1e-6,
)

In [ ]:
dir(res)

In [ ]:
print(f"Suggested Learning Rate: {res.suggestion()}")
res.plot(show=True, suggest=True)
plt.show()

Para el TemporalFusionTransformer, la tasa de aprendizaje óptima es diferente a la sugerida.

A veces, no queremos utilizar directamente la tasa de aprendizaje sugerida porque PyTorch Lightning a veces puede confundirse por el ruido a tasas de aprendizaje más bajas y sugiere tasas demasiado bajas.

El control manual es esencial.

De todas maneras la tasa calculada se encuentra de valores conocidos, por lo tanto la usaremos

In [ ]:
LR = res.suggestion()

In [ ]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard


tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=LR,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    output_size=7,  # 7 quantiles by default
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
    optimizer='adam'
)

trainer = pl.Trainer(
    max_epochs=5,
    accelerator='gpu',   #Descomentar si hay GPU disponible
    devices=1,           #Descomentar si hay GPU disponible
    enable_model_summary=True,
    gradient_clip_val=1,
    #limit_train_batches=50,  # coment in for training, running valiation every 30 batches
    #fast_dev_run=True,  # comment in to check that networkor dataset has no serious bugs
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

In [ ]:
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

In [ ]:
# fit network
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

Vamos a realizar un fine-tuning de hiperparámetros:

**Optuna** es una biblioteca de optimización de hiperparámetros basada en Python diseñada para ser eficiente y flexible. Es particularmente útil en proyectos de machine learning, donde encontrar los mejores hiperparámetros puede ser una tarea compleja y costosa en cuanto a tiempo y recursos.

**Características principales:**

1) Optimización automatizada de hiperparámetros: Optuna permite realizar la búsqueda de hiperparámetros de forma automática y eficiente mediante estrategias de optimización como Tree-structured Parzen Estimator (TPE) y Sequential Halving.

2) Definición de búsqueda sencilla: En Optuna, la búsqueda se estructura en trials (pruebas), y se define una función objetivo que representa la métrica que queremos optimizar (por ejemplo, la precisión o el error de un modelo). Optuna ejecutará esta función varias veces, ajustando los hiperparámetros cada vez.

3) Exploración y explotación inteligente: Usa métodos avanzados que balancean entre probar nuevas configuraciones de hiperparámetros (exploración) y refinar configuraciones prometedoras (explotación), lo que reduce el tiempo y el costo computacional.

4) Soporte para paralelización y multihilo: La biblioteca es compatible con la ejecución de búsquedas en paralelo, lo que permite realizar múltiples pruebas simultáneamente en configuraciones de varios núcleos o en un clúster, mejorando la eficiencia en grandes proyectos.

5) Optimización distribuida: Para proyectos a gran escala, Optuna permite realizar búsquedas distribuidas, lo que facilita su integración en infraestructuras de computación en la nube o sistemas distribuidos.

6) Visualización y seguimiento: Optuna incluye herramientas de visualización para analizar el rendimiento de los trials y entender la convergencia de la optimización. Esto es útil para identificar patrones o configuraciones de hiperparámetros que producen buenos resultados.

In [ ]:
# create study for hyperameter optimization
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=1,
    max_epochs=1,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.0001, 0.1),
    dropout_range=(0.1, 0.3),
    #trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)

# save study results - also we can resume tuning at a later point in time
with open(
    "/home/pdconte/Desktop/Colegio_Bourbaki/ML_AI/Semana2/Data/Model/ModelDatatest_study.pkl",
    "wb",
) as fout:
    pickle.dump(study, fout)

Veamos cuales son los hiperparámetros óptimos:

In [ ]:
# show best hyperparameters
print(study.best_trial.params)

In [ ]:
# load the best model according to the validation loss (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [ ]:
# calculate mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)

In [ ]:
mae_predictions = MAE()(predictions.cpu(), actuals)
mape_predictions = MAPE()(predictions.cpu(), actuals)
smape_predictions = SMAPE()(predictions.cpu(), actuals)
rmse_predictions = RMSE()(predictions.cpu(), actuals)

In [ ]:
#Print mae, mape, smape, rmse errors
print(f'MAE: {mae_predictions}')
print(f'MAPE: {mape_predictions}')
print(f'SMAPE: {smape_predictions}')
print(f'RMSE: {rmse_predictions}')
print(f'MSE: {(rmse_predictions)**2}')

Comparemos con el baseline:

In [ ]:
# Print baseline errors
print("MAE:", mae_baseline_predictions.item())
print("MAPE:", mape_baseline_predictions.item())
print("SMAPE:", smape_baseline_predictions.item())
print("RMSE:", rmse_baseline_predictions.item())
print("MSE:", (rmse_baseline_predictions.item())**2)

Vamos a realizar predicciones:

Después del entrenamiento, podemos hacer predicciones con predict(). El método permite un control muy fino sobre lo que devuelve, de modo que, por ejemplo, puede hacer coincidir fácilmente las predicciones con su marco de datos de pandas. Evaluamos las métricas en el conjunto de datos de validación y en un par de ejemplos para ver lo bien que funciona el modelo.

In [ ]:
# raw predictions are a dictionary from which all kind of information including quantiles can be extracted
raw_predictions = best_tft.predict(val_dataloader, mode="raw", return_x=True)

In [ ]:
for idx in range(10):  # plot 10 examples
    best_tft.plot_prediction(raw_predictions.x, raw_predictions.output, idx=idx, add_loss_to_title=True)

Comprobar cómo se comporta el modelo en distintos segmentos de datos nos permite detectar puntos débiles. Ahora podemos predecir directamente sobre los datos generados utilizando los métodos calculate_prediction_actual_by_variable() y plot_prediction_actual_by_variable(). Las barras grises denotan la frecuencia de la variable por bin, es decir, son un histograma.

In [ ]:
predictions = best_tft.predict(val_dataloader, return_x=True)
predictions_vs_actuals = best_tft.calculate_prediction_actual_by_variable(predictions.x, predictions.output)
best_tft.plot_prediction_actual_by_variable(predictions_vs_actuals)
plt.show()

In [ ]:
interpretation = best_tft.interpret_output(raw_predictions.output, reduction="sum")
best_tft.plot_interpretation(interpretation)
plt.show()

In [ ]:
# Assume raw_predictions has an `encoder_attention` key
encoder_attention = raw_predictions.output.encoder_attention  # Shape should be (batch_size, max_encoder_length, num_attention_heads)

# Average across the batch and attention heads dimensions
mean_encoder_attention_over_time = encoder_attention.mean(dim=(0, 2)).detach().cpu().numpy()

# Create an array for time steps (assuming max_encoder_length is the length of time steps)
time_steps = np.arange(mean_encoder_attention_over_time.shape[0])

# Plot encoder attention over time
plt.figure(figsize=(10, 6))
plt.plot(time_steps, mean_encoder_attention_over_time, label="Encoder Attention")
plt.xlabel("Time Steps (Past Time Indices)")
plt.ylabel("Mean Attention Score")
plt.title("Past Variable Importance Over Time (Using Encoder Attention as Proxy)")
plt.legend()
plt.show()


Ejercicio:

* Realizar predicciones con el modelo entrenado.
* Utilizar la libreria `neuralforecast` de Nixtla para entrenar un nuevo modelo. Inspeccionar sus utilidades.

Link: https://nixtlaverse.nixtla.io/neuralforecast/docs/tutorials/forecasting_tft.html#forecasting-with-tft-temporal-fusion-transformer

### Referencias:

* Introducing PyTorch Forecasting: https://towardsdatascience.com/introducing-pytorch-forecasting-64de99b9ef46

* Demand forecasting with the Temporal Fusion Transformer: https://pytorch-forecasting.readthedocs.io/en/stable/tutorials/stallion.html#Hyperparameter-tuning

* Building Custom Models: https://pytorch-forecasting.readthedocs.io/en/stable/tutorials/building.html#passing-data

![Lenguaje Matemático](./Images/Matematicas.png)

![Contacto](./Images/Contacto.png)